Ãœbung: Canary Deployments
-----------------

In this exercise we will perform a canary deployment.

For this purpose, another deployment is created with the new (Canary) version of the BPMN frontend. The service **doesn't** have a selector for the version, so it controls both versions.

For simplicity, let's use the Blue/Green YAML files with a new service file.

This happens in a separate namespace to be able to display the results in a targeted manner:

In [ ]:
! kubectl create namespace canary

Deployment and Services has been split into separate files. The Ingress resource, omitted for simplicity.

Only the labels of the deployment and of course the image have been extended by one version:

    apiVersion: v1
    kind: Service
    metadata:
      name: bpmn-frontend
      labels:
        app: bpmn-frontend
    spec:
      ...
      selector:
        app: bpmn-frontend
        
- - -

    apiVersion: apps/v1
    kind: Deployment
    metadata:
      name: bpmn-frontend-green
      version: 0.2
    ...
    spec:
      containers:
      - name: bpmn-frontend
        image: registry.gitlab.com/mc-b/misegr/misegr/bpmn-frontend:V0.2
        imagePullPolicy: IfNotPresent   
        
- - -

    apiVersion: apps/v1
    kind: Deployment
    metadata:
      name: bpmn-frontend-blue
      version: 1.0
    ...
    spec:
      containers:
      - name: bpmn-frontend
        image: registry.gitlab.com/mc-b/misegr/misegr/bpmn-frontend:V1.0
        imagePullPolicy: IfNotPresent         

As an exception, we control the number of pods per deployment using `kubectl`.

In [ ]:
! kubectl apply -f 09-4-Deployment/bpmn-frontend-deployment-green.yaml --namespace canary
! kubectl apply -f 09-4-Deployment/bpmn-frontend-deployment-blue.yaml --namespace canary

! kubectl scale --replicas=4 deployment/bpmn-frontend-green --namespace canary
! kubectl scale --replicas=1 deployment/bpmn-frontend-blue --namespace canary

! kubectl apply -f 09-4-Deployment/bpmn-frontend-service-canary.yaml --namespace canary

As a result we have two deployments with five pods. Four with version 0.2 and one with version 1.

In [ ]:
! kubectl get all --namespace canary -o wide
! echo "BPMN Frontend: http://$(cat ~/work/server-ip):$(kubectl get service --namespace canary bpmn-frontend -o=jsonpath='{ .spec.ports[0].nodePort }')/frontend/index.html"

- - -

Clean up

In [ ]:
! kubectl delete namespace canary